In [ ]:
!pip install ultralytics roboflow

In [2]:

from roboflow import Roboflow
rf = Roboflow(api_key="Cf1PZSkmMowyj6foG1Nh")
project = rf.workspace("workspace-zqssx").project("knife-dataset-4kytl")
version = project.version(2)
dataset = version.download("yolov8-obb")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to knife-dataset-2 in yolov8-obb:: 100%|██████████| 8162/8162 [00:01<00:00, 4950.30it/s]


In [ ]:
# prompt: mount the google drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from ultralytics import YOLO
import os
import warnings
import torch


warnings.filterwarnings('ignore', category=FutureWarning)

def train_knife_detector():
    # Get absolute path to the data.yaml file
    script_dir = os.path.dirname(os.path.abspath("__file__"))
    data_yaml_path = os.path.join(script_dir, 'knife-dataset-2', 'data.yaml')

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

      # Initialize a new YOLO model
    model = YOLO('yolov8s.pt')  # Changed to small model for better accuracy

    # Train the model
    results = model.train(
        data=data_yaml_path,
        epochs=100,  # Increased epochs for better learning
        imgsz=640,  # Increased image size
        batch=16,  # Reduced batch size for better generalization
        patience=20,
        device=device,
        project='runs/detect',
        name='cutting_objects_detector',
        # Added training augmentations
        augment=True,
        mixup=0.1,
        mosaic=0.5,
        degrees=10.0,
        translate=0.1,
        scale=0.5,
        shear=2.0,
        perspective=0.0005,
        flipud=0.1,
        fliplr=0.5,
        hsv_h=0.015,  # HSV augmentation
        hsv_s=0.7,
        hsv_v=0.4,
        # Added optimization parameters
        warmup_epochs=3,
        warmup_momentum=0.8,
        box=7.5,  # Box loss gain
        cls=0.5,  # Cls loss gain
        dfl=1.5,  # DFL loss gain
        close_mosaic=10,  # Close mosaic augmentation for last 10 epochs
        label_smoothing=0.1,  # Label smoothing
        overlap_mask=True,  # Overlapping masks
    )

    model.export(format='onnx')

if __name__ == "__main__":
    train_knife_detector()

Using device: cuda
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.64 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/knife-dataset-2/data.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=runs/detect, name=cutting_objects_detector2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, emb

100%|██████████| 755k/755k [00:00<00:00, 21.5MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 84.0MB/s]


AMP: checks passed ✅


...

    100/100      3.84G      1.192     0.5905      1.384         13        640: 100%|██████████| 181/181 [01:04<00:00,  2.80it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:09<00:00,  2.60it/s]

                   all        797        849       0.93      0.879      0.931      0.527



100 epochs completed in 2.197 hours.
Optimizer stripped from runs/detect/cutting_objects_detector2/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/cutting_objects_detector2/weights/best.pt, 22.5MB

Validating runs/detect/cutting_objects_detector2/weights/best.pt...
Ultralytics 8.3.64 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:15<00:00,  1.62it/s]


                   all        797        849      0.898      0.879      0.923       0.53
Speed: 0.2ms preprocess, 12.6ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/cutting_objects_detector2


In [6]:
# prompt: export the model

import os
from google.colab import drive
import locale

# Mount Google Drive
drive.mount('/content/drive')

# **Set the locale to UTF-8**
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Define the source and destination paths
source_model_path = '/content/runs/detect/cutting_objects_detector2/weights/best.pt'  # Replace with the actual path to your exported model
destination_model_path = '/content/drive/MyDrive/postech-hackaton/best.pt'  # Replace with your desired destination path

# Create the destination directory if it doesn't exist
os.makedirs(os.path.dirname(destination_model_path), exist_ok=True)

# Copy the model file to Google Drive
!cp {source_model_path} {destination_model_path}

print(f"Model exported to: {destination_model_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968